In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from cleaning import *

%matplotlib inline

In [5]:
data = pd.read_pickle('../junyi_df.pickle')



In [12]:
data = drop_columns(data)


In [14]:
data = change_bool(data)


In [15]:
convert_dates(data, 'time_done')

In [19]:
#Choosing Date Range
data = select_dates(data, 'time_done', '1/1/2014', '2/1/2014')


In [47]:
data.head()

,user_id,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
68,38355,1,0,0,2014-01-17 03:07:31.028160,2,1,1,0,0,0,225
69,151341,0,0,0,2014-01-21 09:57:31.411600,10,1,1,0,0,0,7
80,47037,1,0,0,2014-01-18 07:53:42.467230,7,1,1,0,0,0,42
93,234664,1,0,0,2014-01-26 11:38:47.448070,16,0,1,1,1,0,0
155,27242,0,0,0,2014-01-04 05:29:19.370960,7,1,1,0,0,0,12


In [32]:
#Creating User Table
users = data[['user_id', 'count_hints']].groupby('user_id').sum()

In [35]:
user_table = pd.DataFrame(users)

In [37]:
user_table

12039

In [38]:
#Creating a More than 10 columns
total_usage = data['user_id'].value_counts().sort_index()

In [50]:
user_table['more_than_ten_activities'] = (total_usage > 10).astype(int)

In [41]:
user_table.head()

,count_hints,more_than_ten_activities
user_id,,
26,0,False
35,18,True
39,0,False
46,0,True
50,0,False


In [ ]:
#Creating a More than 1 day column

In [43]:
user_time = data.groupby('user_id')
rng = (user_time['time_done'].max() - user_time['time_done'].min()).dt.days

In [51]:
user_table['active_more_than_1'] = (rng > 0).astype(int)

In [46]:
user_table.head()

,count_hints,more_than_ten_activities,active_more_than_1
user_id,,,
26,0,False,False
35,18,True,True
39,0,False,False
46,0,True,False
50,0,False,False


In [48]:
#Counting User attempts
user_attempts = data[['user_id', 'count_attempts']].groupby('user_id').sum()
user_table['sum_attempts'] = user_attempts

In [52]:
user_table.head()

,count_hints,more_than_ten_activities,active_more_than_1,sum_attempts
user_id,,,,
26,0,0,0,2
35,18,1,1,155
39,0,0,0,8
46,0,1,0,26
50,0,0,0,1


In [101]:
# Creating Target
target_data = pd.read_pickle('../junyi_df.pickle')
convert_dates(target_data, 'time_done')
target_data = select_dates(target_data, 'time_done', '2/2/2014', '2/9/2014')


# predictor = users['time_done']
# users_ids = users['time_done'].min() < cut_off
# target = predictor[users_ids].astype(int)

In [105]:
target_data.groupby('user_id')

TypeError: 'bool' object is not callable

In [118]:
sorted(list(set(target_data['user_id'])))

[52,
 260,
 327,
 414,
 536,
 797,
 1141,
 1190,
 1267,
 1295,
 1312,
 1378,
 1513,
 1652,
 1668,
 1703,
 1794,
 2274,
 2332,
 2416,
 2653,
 2672,
 2731,
 3079,
 3245,
 3320,
 3474,
 3510,
 3613,
 3703,
 4015,
 4178,
 4215,
 4427,
 4477,
 4485,
 4537,
 4554,
 4589,
 4668,
 4703,
 4709,
 4789,
 4869,
 5323,
 5406,
 5427,
 5669,
 5707,
 5802,
 5966,
 6004,
 6061,
 6821,
 6828,
 6885,
 7020,
 7049,
 7229,
 7238,
 7354,
 7533,
 8019,
 8198,
 8232,
 8488,
 8607,
 8906,
 8909,
 8979,
 8982,
 9103,
 9159,
 9207,
 9268,
 9323,
 9346,
 9426,
 9531,
 9655,
 9873,
 10172,
 10242,
 10274,
 10716,
 10823,
 11294,
 11366,
 11462,
 11474,
 11558,
 11658,
 11780,
 11791,
 12090,
 12141,
 12157,
 12273,
 12510,
 12515,
 12554,
 12671,
 12840,
 12905,
 13158,
 13566,
 13598,
 13656,
 13769,
 13802,
 13895,
 13964,
 14157,
 14225,
 14256,
 14282,
 14404,
 14617,
 14646,
 14756,
 15006,
 15034,
 15112,
 15502,
 15701,
 15899,
 16143,
 16248,
 16354,
 16358,
 16708,
 16712,
 16743,
 16925,
 17218,
 17306,


In [120]:
list(user_table.index)

[26,
 35,
 39,
 46,
 50,
 52,
 53,
 85,
 100,
 109,
 147,
 167,
 213,
 264,
 281,
 289,
 319,
 347,
 350,
 356,
 382,
 434,
 466,
 528,
 533,
 538,
 560,
 608,
 650,
 667,
 675,
 676,
 722,
 723,
 793,
 797,
 825,
 856,
 878,
 906,
 907,
 940,
 1006,
 1010,
 1038,
 1046,
 1085,
 1086,
 1107,
 1114,
 1190,
 1197,
 1211,
 1255,
 1259,
 1267,
 1286,
 1312,
 1314,
 1334,
 1356,
 1368,
 1378,
 1380,
 1396,
 1402,
 1411,
 1420,
 1438,
 1456,
 1467,
 1513,
 1555,
 1583,
 1591,
 1625,
 1642,
 1652,
 1670,
 1673,
 1696,
 1697,
 1739,
 1740,
 1760,
 1769,
 1772,
 1794,
 1838,
 1842,
 1843,
 1855,
 1882,
 1922,
 1979,
 2012,
 2054,
 2068,
 2082,
 2094,
 2113,
 2163,
 2235,
 2328,
 2332,
 2346,
 2376,
 2387,
 2394,
 2403,
 2425,
 2443,
 2447,
 2457,
 2475,
 2480,
 2504,
 2518,
 2545,
 2635,
 2649,
 2653,
 2657,
 2686,
 2709,
 2713,
 2736,
 2744,
 2749,
 2819,
 2841,
 2860,
 2865,
 2877,
 2896,
 2925,
 2950,
 2962,
 2964,
 2968,
 2985,
 2988,
 3000,
 3011,
 3012,
 3043,
 3060,
 3078,
 3079,
 3105,


In [ ]:
target = dict()

for user in list(user_table.index):
    if user in set(target_data['user_id']):
            target[user] = 1
    else: 
            target[user] = 0

In [96]:
target_data['user_id'].values

array([], dtype=int64)

In [95]:
target_users

set()

In [ ]:
#Getting last ten 10

In [65]:
user_indexed = data.set_index('user_id').sort_values(by='time_done')

In [66]:
user_indexed.head()

,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
user_id,,,,,,,,,,,
237856,0,0,0,2014-01-01 00:00:03.148100,6,1,1,0,0,0,75
154395,0,0,1,2014-01-01 00:00:14.107860,22,1,1,0,0,0,5
119799,1,1,0,2014-01-01 00:00:15.110180,10,1,1,0,0,0,225
154395,0,0,1,2014-01-01 00:00:19.035330,4,1,1,0,0,0,14
119799,1,1,0,2014-01-01 00:00:24.901530,5,1,1,0,0,0,225


In [67]:
user_indexed.loc[35, :]

,topic_mode,suggested,review_mode,time_done,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,points_earned
user_id,,,,,,,,,,,
35,0,0,0,2014-01-08 06:43:24.504230,255,0,7,1,2,0,0
35,0,0,0,2014-01-09 09:01:02.373640,60,0,3,0,0,0,0
35,0,0,0,2014-01-09 09:02:21.835700,45,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:02:40.267920,11,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:02:50.194020,18,0,2,0,0,0,0
35,0,0,0,2014-01-09 09:03:05.104680,13,0,2,0,0,0,0
35,0,0,0,2014-01-09 09:03:24.794640,14,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:03:34.756280,8,1,1,0,0,0,75
35,0,0,0,2014-01-09 09:03:46.724700,25,0,3,0,0,0,0


In [81]:


user_indexed.loc[35, 'count_attempts'][-10:].sum()

56

In [79]:

y = user_indexed.loc[35, 'count_hints'][-10:].sum()

In [80]:
y

8

In [ ]:
for user_id in set(new_data.index.values):
#     days_active[user_id] = get_active_weeks(user_id)

In [ ]:
def get_last_10_hints(user_id):
    try: 
        return len(set(new_data.loc[user_id, 'week']))
    except:
        return 0

In [ ]:
#Creating a More than 1 day co